# Python for (open) Neuroscience

_Lecture 1.1_ - More on `numpy`

Luigi Petrucco

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec-2024/blob/main/lectures/Lecture1.1_Numpy.ipynb)

## Transforming and combining arrays

### `.T`

We can get a **transposed view** (NOT A COPY) of a matrix with the `.T` attribute:

In [20]:
m = np.ones((3,2))
print(m)

[[1. 1.]
 [1. 1.]
 [1. 1.]]


In [21]:
m_t = m.T
print(m_t)

[[1. 1. 1.]
 [1. 1. 1.]]


In [22]:
m_t[0, 0] = 2  # this is a view: if we change values in the transposed array, we change the original as well:
m

array([[2., 1.],
       [1., 1.],
       [1., 1.]])

### `.flatten()`

We can flatten all values of an N-dimensional array into a 1D array with the `.flatten()` syntax. This will make **a copy of the array**!

In [11]:
m = np.ones((3,2,3))
m_flat = m.flatten()

print(m.shape, m_flat.shape)
m_flat

(3, 2, 3) (18,)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

### `np.concatenate()`

We can concatenate arrays along any dimension by putting them in a list and pass the list to the `np.concatenate()` function:

In [135]:
arr_list = [np.zeros(3), np.ones(3)]

np.concatenate(arr_list) 

array([0., 0., 0., 1., 1., 1.])

By default, we concatenate over the first dimension:

In [136]:
arr_list = [np.zeros((2,3)), np.ones((1,3))]
arr_list

[array([[0., 0., 0.],
        [0., 0., 0.]]),
 array([[1., 1., 1.]])]

In [137]:
np.concatenate(arr_list)  # if ndims > 1 by default we concatenate over the first dimension

array([[0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.]])

but we can pass an `axis` argument to change the default behavior:

In [138]:
arr_list = [np.zeros((3,2)), np.ones((3,2))]

np.concatenate(arr_list, axis=1) 

array([[0., 0., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 1., 1.]])

### `np.stack()`

We can pile up arrays over a new dimension with  `np.stack()`:

In [ ]:
arr_list = [np.zeros((3, 2)), np.ones((3, 2))]

np.stack(arr_list).shape

## Array operations

We obviously want to do some math with those arrays!

### Operations with numbers

Operations with arrays are **by default element-wise**! (performed separately on every number of the array)

Sum / subtraction / multiplication / division apply to individual entries of the array:

In [139]:
np.ones(3) + 1

array([2., 2., 2.])

In [143]:
my_arr = np.ones((4,3))
my_arr[0, :] *= 100  # syntax equivalent to my_arr[0, :] = my_arr[0, :] * 100
my_arr

array([[100., 100., 100.],
       [  1.,   1.,   1.],
       [  1.,   1.,   1.],
       [  1.,   1.,   1.]])

Exponentiation also works element-wise:

In [144]:
np.array((1,2,3))**3

array([ 1,  8, 27])

### Operations between arrays

`numpy` works element-wise also when operating between arrays:

In [147]:
arr_1 = np.array([[1,2],
                  [3,4]])
arr_2 = np.array([[0,0],
                  [0,2]])

arr_1 * arr_2

array([[0, 0],
       [0, 8]])

In [148]:
arr_1 ** arr_2

array([[ 1,  1],
       [ 1, 16]])

Therefore, we normally expect arrays of matching shapes, or we get a `ValueError`!

In [149]:
np.ones((2, 3)) * np.ones((4, 5))

ValueError: operands could not be broadcast together with shapes (2,3) (4,5) 

### Broadcasting

`numpy` has a smart way of dealing with some scenarios of non-matching dimensions, and we should use it!

Can be a bit tricky at the beginning, but it is very important: we can write very efficient and readable code with it!

In [152]:
# Assume we have a matrix of data:
data = np.array([[ 0.0,  0.0,  0.0],
                 [10.0, 10.0, 10.0],
                 [20.0, 20.0, 20.0],
                 [30.0, 30.0, 30.0]])

data.shape

(4, 3)

In [153]:
offsets = np.array([1.0, 2.0, 3.0])  # we want to add an offset to each data column:
data + offsets

array([[ 1.,  2.,  3.],
       [11., 12., 13.],
       [21., 22., 23.],
       [31., 32., 33.]])

### What is happening?

Numpy automatically infer missing values to create arrays of matching shape, where it can the operate element-wise!

![Alt Text](https://numpy.org/doc/stable/_images/broadcasting_2.png)

## How does broadcasting work

When operating on two arrays, NumPy compares their shapes. It starts **with the trailing** dimension - i.e., the rightmost dimension in the `shape` tuple - and works its way left. In a `(2,3)` matrix, broadcasting will start from columns (the `3` in `(2,3)`) and move on to rows (the `2` in `(2,3)`).

Two dimensions are compatible when:

 - they are equal, or
 - one of them is 1.

![Alt Text](https://i0.wp.com/andrewm4894.com/wp-content/uploads/2020/10/Annotation-2020-10-15-133235.jpg?w=486&ssl=1)

In our case:

In [ ]:
print(f"shape a: {a.shape}")
print(f"shape b: {b.shape}")

Shape b matches shape a over the last dimension, and is propagated over the rest of the dimensions

For example, this operation will not work!

In [28]:
a = np.concatenate([np.full((1,4), i) for i in range(5)], axis=0)
print(a)

[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]]


In [29]:
b = np.ones(5)
print("shape a: ", a.shape)
print("shape b: ", b.shape)

# this will not work as the rightmost dimensions are 5 and 4:
a + b 

shape a:  (5, 4)
shape b:  (5,)


ValueError: operands could not be broadcast together with shapes (5,4) (5,) 

To make it work, we can use a trick: add a new "dummy" singleton dimension to `b` that will be broadcasted with the syntax `[:, np.newaxis]`

In [30]:
b_twodim = b[:, np.newaxis]  # This does the trick by adding a dummy singleton dimension
print("shape a: ", a.shape)
print("shape b_twodim: ", b_twodim.shape)
a + b_twodim  # now the last dimension is compatible between the two arrays:

shape a:  (5, 4)
shape b_twodim:  (5, 1)


array([[1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.],
       [4., 4., 4., 4.],
       [5., 5., 5., 5.]])

Practicals 1.1.0

## More on `numpy` functions

### `np.mean()` / `np.nanmean()`

Calculate the average of an array, either global or along some axis:

In [152]:
import numpy as np
arr = np.random.randint(0, 255, (5, 6))
mean = np.mean(arr)  # compute the mean of the whole array

print(f"{arr};\nmean: {mean}")

[[ 48 170  87 127  88  92]
 [126  85 182 253 219  67]
 [179 117 230 149 114 213]
 [ 59 169 190 128  57  91]
 [ 17  78 184 143 249 174]];
mean: 136.16666666666666


If we want to take the average along a specific dimension, we can pass the axis as a parameter:

In [9]:
import numpy as np
arr = np.random.randint(0, 255, (3, 6))
arr_mean = np.mean(arr, axis=1)  # we specify one axis along which to mean

print(f"{arr};\nmean: {arr_mean}")

[[ 21 119  79 247 244 156]
 [  6  30  88 213 228  12]
 [ 57  27 197 151 210 110]];
mean: [144.33333333  96.16666667 125.33333333]


If there are nan values around, we have to use `np.nanmean()`:

In [154]:
import numpy as np

# we need a float dtype to use nan values:
arr = np.random.randint(0, 255, (5, 6)).astype(float) 

arr[0, 0] = np.nan
arr_mean = np.mean(arr)  # regular mean
arr_nan_mean = np.nanmean(arr)  # use nanmean

print(f"{arr};\nregular mean: {arr_mean}\nnanmean: {arr_nan_mean}")

[[ nan  88. 227.  96. 128. 145.]
 [143. 199.  86. 221.  50.  14.]
 [195.  15. 189. 184.  31. 133.]
 [ 21. 194.  83. 185.  35. 218.]
 [143. 237. 100. 168.  56.  82.]];
regular mean: nan
nanmean: 126.41379310344827


Many of the functions we're about to see behave in this way - assume they have a nan-dealing equivalent!

 - `np.std()` / `np.nanstd()`
 - `np.percentile()` / `np.nanpercentile()`
 - `np.max()` / `np.nanmax()`
 - ...

### `np.std()` / `np.nanstd()`

Calculate the standard deviation of an array, either global or along some axis:

In [23]:
arr = np.random.normal(0, 3, 1000)
np.std(arr)

2.9911940005367863

### `np.median()` / `np.nanmedian()`

Calculate the median of an array, either global or along some axis:

In [27]:
arr = np.random.randint(0, 100, (1000, 10))
np.median(arr, axis=0)

array([51., 49., 49., 49., 51., 50., 49., 47., 49., 53.])

### `np.max()` / `np.min()`

Calculate the minimum or maximum of an array, either global or along some axis:

In [28]:
arr = np.random.randint(0, 100, 1000)

np.min(arr), np.max(arr)  # print min and max together

(0, 99)

### `np.percentile()`

Calculate a given percentile of an array, either global or along some axis:

In [29]:
arr = np.random.randint(0, 1000, 10000)

np.percentile(arr, 75)  # print min and max together

756.0

### `np.unique()`

Return unique values of an array, and if asked their counts

In [33]:
np.unique(np.array([1, 2, 2, 3, 3, 3]))

array([1, 2, 3])

In [35]:
# If we ask we can get counts as well
unique_values, counts = np.unique([1,2,2,3], return_counts=True) 

print("unique:", unique_values)

print("counts:", counts)

unique: [1 2 3]
counts: [1 2 1]


In [36]:
arr = np.random.normal(0, 1, 1000000)

### `np.diff()` / `np.cumsum()`  

We can compute cumulative sums (integrals) of an array with `np.cumsum()`:

In [39]:
my_arr = np.array([[1,2,3,4],[1,2,4,5]])
np.cumsum(my_arr, axis=0)

array([[1, 2, 3, 4],
       [2, 4, 7, 9]])

We can compute differences between consecutive elements of an array using `np.diff()`:

In [40]:
my_arr = np.array([1,2,3,4])
np.diff(my_arr)

array([1, 1, 1])

(Practicals 1.1.1)

### Write code the `numpy` way

When operating with matrices, you should always aim at writing <span style="color:indianred">vectorized</span> code

Vectorized code: code where for loops are replaced by operations over matrix dimensions

An very simple example: vectors multiplication

In [41]:
vector_1 = np.random.normal(0, 1, (10000000,))
vector_2 = np.random.normal(0, 1, (10000000,))

In [42]:
%%timeit
product = np.zeros(vector_1.shape)  # initialize empty result vector

# Compute the multiplication in a loop:
for i in range(vector_1.shape[0]):
    product[i] = vector_1[i] * vector_2[i]

1.54 s ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
product = vector_1 * vector_2

7.77 ms ± 21.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Another example: Z-score the rows of a matrix:

In [5]:
data_matrix = np.random.randint(0, 255, (100000, 100))

In [6]:
%%timeit
normalized_matrix = np.zeros(data_matrix.shape)  # start an empty matrix of matching shape 

# Loop over rows (first dimension), take mean and std, subtract and divide:
for i in range(data_matrix.shape[0]):
    row_mean = np.mean(data_matrix[i, :])
    row_std = np.std(data_matrix[i, :])
    
    normalized_matrix[i, :] = (data_matrix[i, :] - row_mean) / row_std

1.3 s ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
rows_mean = np.mean(data_matrix, axis=1)  # vectorized mean
rows_std = np.std(data_matrix, axis=1)  # vectorized std

# Now we write the normalization as a vector operation.
# Note how we use broadcasting to make sure the right dimensions are propagated!
normalized_matrix = data_matrix - rows_mean[:, np.newaxis] / rows_std[:, np.newaxis]

43.4 ms ± 918 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
rows_mean.shape

(100000,)

## Search indexes

Some functions find indexes of the elements of an array that match some criterion.

### `np.argmin()` / `np.argmax()` 

Find the position of the maximum or the minimum of an array

In [60]:
arr = np.array([5,  0, 2, 9, 6,])

np.argmin(arr)  # give index of smallest element

1

In [61]:
np.argmax(arr)  # give index of biggest element

3

### `np.nonzero()` / `np.argwhere()`

There are functions that allow us to get the index of all True elements in an array. In this way, we can set a condition (_e.g._, values above a threshold), and find the index of all elements satisfying it!

#### `np.argwhere()`

`np.argwhere()` find `True` elements and gives us a `(n_pts, n_matrix_dims)` shaped array of indexes:

In [12]:
arr = np.array([[1, 2, 3, 4, 5], 
                [0, 5, 0, 3, 1],
                [0, 6, 7, 4, 1]])  # define an array
boolean_vals = arr > 2  # boolean condition
print("Original array:")

print(arr)
print("\nBoolean array:")
print(boolean_vals)

Original array:
[[1 2 3 4 5]
 [0 5 0 3 1]
 [0 6 7 4 1]]

Boolean array:
[[False False  True  True  True]
 [False  True False  True False]
 [False  True  True  True False]]


In [13]:
indexes = np.argwhere(boolean_vals)
print("\nTrue elements indexes:")
print(indexes)


True elements indexes:
[[0 2]
 [0 3]
 [0 4]
 [1 1]
 [1 3]
 [2 1]
 [2 2]
 [2 3]]


The indexes matrix has one row for every True value, and each column represents the position of that value on the boolean matrix.

#### `np.nonzero()`

`np.nonzero()` is very similar to `np.argwhere()`, but instead of a single matrix of indexes (with each column representing the indexes over one dimension for all points), it returns us a tuple of arrays. 

That is to say, each one of those arrays corresponds to a column of the indexes array returned by `np.argwhere()`.

In [14]:
# print("\nBoolean array:")
#print(boolean_vals)

indexes_tuple = np.nonzero(boolean_vals)
# print("\nTrue elements indexes:")
print(indexes_tuple)

(array([0, 0, 0, 1, 1, 2, 2, 2]), array([2, 3, 4, 1, 3, 1, 2, 3]))


indexes_tuples is a tuple of arrays; each one of thise array corresponds to a column in the indexes matrix above!

In [141]:
indexes_tuple[0] == indexes[:, 0]

array([ True,  True,  True,  True,  True,  True,  True,  True])

Why is it useful to return a tuple of arrays?

### Array indexes are tuples!

Whenever you are writing something like:

In [142]:
arr[0, 1]

2

You are actually passing a tuple into the square brackets! If you remember, in python, comma separated
values with no brakets are automatically put together in a tuple:

In [143]:
a = 1, 2
type(a)

tuple

So, writing `arr[0, 1]` is literally the same as writing `arr[(0, 1)]`:

In [144]:
arr[(0, 1)]

2

If you remember, you can pass arrays instead of single numbers for indexing:

In [150]:
# This code will retrieve three elements from arr:
#   - the element in (0, 2)
#   - the element in (1, 3)
#   - the element in (1, 1)

print(arr)
print("\nSelected items:")
arr[np.array([0, 1, 1]), np.array([2, 3, 1])]

[[1 2 3 4 5]
 [0 5 0 3 1]
 [0 6 7 4 1]]

Selected items:


array([3, 3, 5])

Therefore, we can directly use the indexes arrays tuple we get from `np.nonzero()` to retrieve all elements of the original matrix that matched the boolean condition!

In [16]:
arr = np.array([[1, 2, 3, 4, 5], 
                [0, 5, 0, 3, 1],
                [0, 6, 7, 4, 1]])
boolean_vals = arr > 2
print("Original array:")
print(arr)
print("\nBoolean array:")
print(boolean_vals)

indexes_tuple = np.nonzero(boolean_vals)
indexes_tuple

Original array:
[[1 2 3 4 5]
 [0 5 0 3 1]
 [0 6 7 4 1]]

Boolean array:
[[False False  True  True  True]
 [False  True False  True False]
 [False  True  True  True False]]


(array([0, 0, 0, 1, 1, 2, 2, 2]), array([2, 3, 4, 1, 3, 1, 2, 3]))

In [17]:
filtered_values = arr[indexes_tuple]  # here we directly use the tuple to index the array!
print("\nElements bigger than 2:")
print(filtered_values)


Elements bigger than 2:
[3 4 5 5 3 6 7 4]


(Practicals 1.1.2)

## [Bonus] index raveling / unraveling

For a multi-dimensional array:

In [90]:
arr = np.array([[5, 1, 2], 
                [3, 0, 4]])
print(arr)
np.argmin(arr)

[[5 1 2]
 [3 0 4]]


4

What is this number 4?

## Flat indexing

When you have a multi-dimensional array, you can always index it in two ways:
 - the standard, multi-dimensional indexing (e.g., `my_array[3,4]`)
 - with **flat indexing**: we index the array after flattening it out in a single dimension

In [4]:
# Example:
my_arr = np.random.randint(0, 10, (4, 3))
my_arr

array([[8, 3, 1],
       [8, 2, 9],
       [9, 1, 1],
       [5, 5, 8]])

There is a flatten representation of this array that we can look at with `.flatten()` method we saw in the last lecture:

In [10]:
import numpy as np
# When we flatten, we concatenate all values of the matrix in a single dimension.
# We keep the order of the dimensions of the matrix (the first 3 elements are the first row, that
# is, the first dimension):
print(my_arr)
min_idx = np.argmin(my_arr)
print(min_idx)
my_arr.flatten()[min_idx]


[[8 3 1]
 [8 2 9]
 [9 1 1]
 [5 5 8]]
2


1

The number we got from `np.argmax()` is the number we would need to use over the flattened representation of the array to get the maximum value!

In [118]:
max_idx = np.argmax(my_arr)  # get max index
print(max_idx)

flat_array = my_arr.flatten()  # create a flattened array
flat_array[max_idx]

2


7

One last thing. it is obviously annoying having to create a new flattened array to use the index. Also, we create a duplicated array in memory - not good.

The best way to use this indexing it through the `.flat` representation of the matrix:

In [117]:
my_arr.flat[max_idx]

7

### Index raveling / unraveling

To convert the flat index to a tuple of matrix indexes, we can use `np.unravel_index()`:

In [124]:
arr = np.array([[5, 1, 2], [3, 3, 0]])
idx = np.argmin(arr)

# unravel index takes an index of the flattened array, and the shape of the matrix,
# and give us the tuple of ordinary indexes that correspond to that flat index.
np.unravel_index(idx, arr.shape)  

(1, 2)

This is an illustration of what happens. Flat indexes (_left_) are converted to tuple indexes (_right_):
(there is a bug in the image, last value should be 11!)

![unravel illustration](https://i.stack.imgur.com/sxwBU.png)

The converse operation, called unravel, can be done with `np.ravel_multi_index()`, and it goes from the right representation to the left one:

In [125]:
np.ravel_multi_index((1, 1), arr.shape)

4